In [70]:
import requests
from urllib import parse
import time
from datetime import datetime

import pandas as pd
import os

In [78]:
def binarySearchAsc(mItem, mList):
    
    startIndex = 0
    endIndex = len(mList) - 1
    
    while(startIndex <= endIndex):
        target = int((startIndex + endIndex) / 2)
        
        if(mItem == mList[target]):
            return [True, target]
        elif(mItem < mList[target]):
            endIndex = target - 1
        else:
            startIndex = target + 1
            
    if(endIndex == -1):
        return [False, startIndex]
    if(startIndex > endIndex):
        return [False, startIndex]
    return [False, target]

In [17]:
with open('apiKey.txt', 'r') as f:
    apiKey = f.read()
requestHeader = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0",
    "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-4c9fee46-1933-462b-9183-d6eee9a06b85"
}

In [33]:
url = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + "hideonbush"
result = requests.get(url, headers=requestHeader)

In [35]:
puuid = result.json()['puuid']

In [37]:
url = "https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/" + puuid + "/ids?start=0&count=100"
result = requests.get(url, headers=requestHeader)

In [42]:
matchid = result.json()[0]

In [43]:
url = "https://asia.api.riotgames.com/lol/match/v5/matches/" + matchid
result = requests.get(url, headers=requestHeader)

In [59]:
#puuid
result.json()['metadata']['participants'][0]

'DmVbHxri4kdPllcuGFjibJzRjFkl66pGG5geTmWuhyPAI0aIIFbQfTGO02ecrBR4F7PfoicGFOqCxQ'

In [76]:
ts = int(result.json()['info']['gameCreation'])
ts

1657040888000

In [56]:
puuidList = []
matchList = [[], []]

#csv
puuidDf = pd.read_csv('data/puuid.csv')
matchdf = pd.read_csv('data/match.csv')

#oracle


i0 = 0
while(i0 < len(puuidList)):
    url = "https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/" + puuidList[i0] + "/ids?start=0&count=100"
    puuidResult = requests.get(url, headers=requestHeader)
    
    bool readAll = False
    
    for matchid in puuidResult.json():
        searchMatch = binarySearchAsc(matchid, matchList[0])
        if searchMatch[0] == True:
            continue
        
        url = "https://asia.api.riotgames.com/lol/match/v5/matches/" + matchid
        matchResult = requests.get(url, headers=requestHeader)
        
        for puuid in matchResult.json()['metadata']['participants']:
            searchPuuid = binarySearchAsc(puuid, puuidList)
            if searchPuuid[0] == True:
                continue
                
            puuidList.insert(searchResult[1], puuid)
        
        matchList[0].insert(searchMatch[1], matchid)
        matchList[1].insert(searchMatch[1], matchResult)
        